# Layers and Blocks

To begin, we revisit the code
that we used to implement MLPs

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 4.0005e-02, -1.4787e-01, -5.1628e-02,  1.2147e-01,  1.3309e-01,
         -3.7183e-02, -1.0823e-01, -1.2697e-02, -6.8065e-02, -1.1334e-02],
        [-2.0080e-02, -1.6836e-01, -1.1021e-01,  8.8831e-02,  4.2754e-02,
         -1.3104e-02, -4.8720e-02, -8.3985e-02,  1.0065e-04, -9.4423e-02]],
       grad_fn=<AddmmBackward>)

`nn.Sequential` defines a special kind of `Module`

A Custom Block

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

Instantiate the MLP's layers
and subsequently invoke these layers

In [3]:
net = MLP()
net(X)

tensor([[ 0.4992, -0.2680, -0.0344,  0.0066,  0.1405,  0.1018, -0.1287, -0.0794,
         -0.1635,  0.0661],
        [ 0.4407, -0.2366, -0.0346,  0.0181,  0.1552,  0.0708, -0.2082, -0.0799,
         -0.0441,  0.1757]], grad_fn=<AddmmBackward>)

The Sequential Block

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1430, -0.0816,  0.2252, -0.0530, -0.2040,  0.2465, -0.1851,  0.1464,
          0.0046, -0.0960],
        [ 0.1906, -0.0608,  0.3937, -0.1902, -0.2266,  0.1863, -0.1154,  0.1527,
         -0.1074, -0.1636]], grad_fn=<AddmmBackward>)

Executing Code in the Forward Propagation Function

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.2127, grad_fn=<SumBackward0>)

Mix and match various
ways of assembling blocks together

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1049, grad_fn=<SumBackward0>)